In [3]:
import pandas as pd
import numpy as np
# import embedder as emb

In [6]:
csv_path = "/home/ale/Desktop/Vinted-Web-Scraper/big_csv/big_csv.csv"
big_df = pd.read_csv(csv_path)


<ipython-input-6-b0411f0ba162>:2: DtypeWarning: Columns (1,5,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  big_df = pd.read_csv(csv_path)


# Fix period in prices


In [ ]:


def fix_price(price):
    if isinstance(price, str) and '.' not in price and price.isdigit():
        return f"{price[:-2]}.{price[-2:]}"  # Insert period before the last two digits
    return price

# Apply the function to the 'Price' column
big_df['Price'] = big_df['Price'].apply(fix_price)

# Drop columns

In [ ]:
columns_to_drop = ["Page", "SearchCount", "Condition", "LastCheck", "Dataid", "Link", "" ]
big_df = big_df.drop(columns=columns_to_drop)

# Parse size to float

In [ ]:
def float_size(price):
    if isinstance(price, int):
        return float(price)
    return price

big_df["Size"] = big_df["Size"].apply(float_size)

# Remove sold items from big csv

In [11]:
sold_csv_path = "/home/ale/Desktop/Vinted-Web-Scraper/sold_items.csv"
sold_df = pd.read_csv(sold_csv_path)


df_result = pd.merge(big_df, sold_df, on="Dataid", how='left', indicator=True).query('_merge == "left_only"').drop(columns='_merge')

print(len(df_result))

55114


# Cast Market Status 

In [ ]:
big_df['MarketStatus'] = 0

# Move 'MarketStatus' column to the end
market_status = big_df.pop('MarketStatus')
big_df['MarketStatus'] = market_status

In [ ]:
sold_descriptions = df_final[df_final['MarketStatus'] == 1]['Description'].tolist()
not_sold_descriptions = df_final[df_final['MarketStatus'] == 0]['Description'].tolist()

In [ ]:
# Getting embeddings
sold_embeddings = [emb.get_embedding(sentence) for sentence in sold_descriptions]
not_sold_embeddings = [emb.get_embedding(sentence) for sentence in not_sold_descriptions]

# Calculating average locations (centroids)
sold_centroid = emb.calculate_centroid(sold_embeddings)
not_sold_centroid = emb.calculate_centroid(not_sold_embeddings)